In [ ]:
import gspread
import serial
import time
from serial.tools import list_ports
from oauth2client.service_account import ServiceAccountCredentials

# Set your credentials JSON file path and Google Sheet name
CREDENTIALS_FILE = 'automated-hydroponics-data-40ded0143ddf.json'
SPREADSHEET_NAME = 'Test'
WORKSHEET_NAME = 'Sheet1'

# Authenticate using the JSON file
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, scope)
client = gspread.authorize(creds)

# Open the spreadsheet
spreadsheet = client.open(SPREADSHEET_NAME)

# Select the specific worksheet by name
worksheet = spreadsheet.worksheet(WORKSHEET_NAME)

# List available serial ports
ports = list_ports.comports()
for port in ports:
    print(port)

# Open a serial connection
serialCom = serial.Serial("COM9", 9600)

# Prepare the serial connection
serialCom.setDTR(False)
time.sleep(1)
serialCom.flushInput()
serialCom.setDTR(True)

while True:
    try:
        # Read a line of data from the serial connection
        s_bytes = serialCom.readline()
        decoded_bytes = s_bytes.decode("utf-8").strip("\n\r")

        # Split the received data into individual values
        lines = decoded_bytes.split(',')
        EC = float(lines[0])
        pH = float(lines[1])
        Humidity = float(lines[2])
        Air_temperature = float(lines[3])
        Water_temperature = float(lines[4])
        Distance = float(lines[5])

        # Print and send EC/2 data to the Google Sheet
        print(EC / 2)
        data_to_send = [EC / 2]  # Send EC/2 to the Google Sheet
        worksheet.append_row(data_to_send)

    except Exception as e:
        print("ERROR! There was an error in the code:", str(e))